In [1]:
import backtrader as bt
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [88]:
class VWAPStrategy(bt.Strategy):
    params = (
        ("ema_period", 50),     # EMA Period
        ('atr_period', 9) ,      # ATR Period
        ('atr_multiplier', 1.5), # ATR multiplier for SL calculation
        ("pivot_window", 5),  # 5 bars for pivot high/low
        ("rrr", 2),  # Risk-Reward Ratio
    )
    def __init__(self):
        # initialize the EMA indicator on data feed
        # self.ema = bt.talib.EMA(self.data.close, self.params.ema_period)
        self.ema = bt.indicators.ExponentialMovingAverage(period=self.params.ema_period)
        self.atr = bt.indicators.AverageTrueRange(period=self.params.atr_period)
        self.crossover = bt.indicators.CrossOver(self.data.close, self.ema)
        self.order = None
        self.prev_pivot_high = self.data.low[0]
        self.prev_pivot_low = self.data.high[0]
        
    def _isBulluish(self, idx:int):
        if len(self.data) < idx+1:
            self.log("[Error]: Index out of bounds for `isBullish`")
            return
        return self.data.close[idx] > self.data.open[idx]

    def _isEngulfing(self, idx:int):
        if len(self.data) < idx+1:
            self.log("[Error]: Index out of bounds for `isEnglufing`")
            return
        return self.data.open[idx+1] < self.data.close[idx] if self._isBulluish(idx) \
            else self.data.open[idx+1] > self.data.close[idx]

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.order = None
    
    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
    
    
    def _update_pivots(self):
        if np.max(self.data.high.get(size=self.params.pivot_window * 2 + 1)) == self.data.high[self.params.pivot_window]:
            self.prev_pivot_high = self.data.high[self.params.pivot_window]   
        if np.min(self.data.low.get(size=self.params.pivot_window * 2 + 1)) == self.data.low[self.params.pivot_window]:
            self.prev_pivot_low = self.data.low[self.params.pivot_window]      

    def next(self):
        # if not in a market position, check for crossover from below
        if self.position:
            return
        if self.crossover[0] > 0 and (self._isEngulfing(0) or self._isEngulfing(1)):
            price = self.data.close[0]
            sl_price = price - self.atr * self.params.atr_multiplier
            tp_price = price + np.abs(sl_price - price) * self.params.rrr
            if tp_price < price or tp_price < 0:
                self.log(f'Take profit price {tp_price:.2f} is less than entry price {price:.2f}, not placing order.')
                return
            if sl_price > price or sl_price < 0:
                self.log(f'Stop loss price {sl_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                return

            self.order = self.buy_bracket(
                exectype=bt.Order.Market, 
                price=price,
                stopprice=sl_price,
                limitprice=tp_price,
                stopargs=dict(exectype=bt.Order.Stop),
                limitargs=dict(exectype=bt.Order.Limit),
            )
            self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")

        # if in a position, check for crossover from above for exit
        elif self.crossover[0] < 0 and (self._isEngulfing(0) or self._isEngulfing(1)):
            price = self.data.close[0]
            sl_price = price + self.atr * self.params.atr_multiplier
            tp_price = price - np.abs(sl_price - price) * self.params.rrr
            if tp_price > price or tp_price < 0:
                self.log(f'Take profit price {tp_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                return
            if sl_price < price or sl_price < 0:
                self.log(f'Stop loss price {sl_price:.2f} is less than entry price {price:.2f}, not placing order.')
                return
            self.order = self.sell_bracket(
                exectype=bt.Order.Market, 
                price=price,
                stopprice=sl_price,
                limitprice=tp_price,
                stopargs=dict(exectype=bt.Order.Stop),
                limitargs=dict(exectype=bt.Order.Limit),
            )
            self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")        


## Source data locally

In [89]:
data_local = pd.read_csv("../data/xauusd_1h.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

             datetime         open         high          low        close  \
0 2010-01-03 22:00:00  1097.880005  1097.880005  1097.449951  1097.449951   
1 2010-01-03 23:00:00  1097.680054  1100.599976  1095.979980  1095.979980   
2 2010-01-04 00:00:00  1096.010010  1096.869995  1093.449951  1094.839966   
3 2010-01-04 01:00:00  1094.869995  1095.959961  1094.239990  1095.699951   
4 2010-01-04 02:00:00  1095.670044  1099.150024  1095.630005  1098.329956   

   volume  
0       2  
1     919  
2    1596  
3     869  
4    1054  


## Download Data from YF

In [95]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2023-06-01', 'end': '2025-01-01',
             'interval': '1h', 'multi_level_index': False, 'auto_adjust': True}
data_names = ['^RUT', '^IXIC', '^DJI', '^GSPC', '^VIX', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 
              'NFLX', 'NVDA', 'AMD']

datas = dict()
for name in data_names:
    datas[name] = yf.download(tickers=name, **yf_params)


for name, data in datas.items():
    data.reset_index(inplace=True)
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    print(f"{name}: from {data['Datetime'].iloc[0]} to {data['Datetime'].iloc[-1]}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

^RUT: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
^IXIC: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
^DJI: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
^GSPC: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
^VIX: from 2023-06-01 07:00:00+00:00 to 2024-12-31 21:00:00+00:00
GOOG: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
MSFT: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
AMZN: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
TSLA: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
NFLX: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
NVDA: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00
AMD: from 2023-06-01 13:30:00+00:00 to 2024-12-31 20:30:00+00:00


In [96]:
data_feeds = dict()
for name, data in datas.items():
    data_feeds[name] = bt.feeds.pandafeed.PandasData(
        name=name,
        dataname=data,
        datetime=0,
        openinterest=-1,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
    )

# Start Backtesting

In [97]:
def log_results(data_name:str, results) -> None:
    print("\n\n" + 20*"=" + f" {data_name} " + 20*"=")
    strat = results[0]
    sharpe_ratio = strat.analyzers.sharperatio.get_analysis()
    drawdown = strat.analyzers.drawdown.get_analysis()
    trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()
    returns = strat.analyzers.returns.get_analysis()
    print(f'Sharpe Ratio: {sharpe_ratio['sharperatio']}')
    print(f'Max Drawdown: {drawdown["max"]["drawdown"]}')
    print(f'Number of Trades: {trade_analysis.total.closed}')
    print(f'Winning Trades: {trade_analysis.won.total}')
    print(f'Losing Trades: {trade_analysis.lost.total}')
    print(f'Average Trade Return: {returns["rnorm"]}')
    print(f'Total Returns: {returns["rtot"]}')
    print(f'Final Balance: {strat.broker.getvalue()}')

In [105]:
def test_strategy(data_feed, strategy_params):
    cerebro = bt.Cerebro()
    # Add the Strategy
    cerebro.addstrategy(VWAPStrategy, 
                        **strategy_params
                        )
    # Add trade size manager (Percent of equity)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=5)

    cerebro.adddata(data_feed)  # Add DataFeed
    # Set cash and commission
    cerebro.broker.setcash(100_000.0)
    cerebro.broker.setcommission(commission=0.001)
    # Add Analyzers for final review
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
    cerebro.addanalyzer(bt.analyzers.Returns)

    results = cerebro.run(stdstats=False)
    
    return results, cerebro

In [108]:
strategy_params = dict(ema_period=21,
                       atr_period=9,
                       atr_multiplier=1.5,
                       rrr=1.5,
)
results = dict()

for name, data_feed in data_feeds.items():
    results[name], cereb = test_strategy(data_feed, strategy_params)
    # cereb.plot(iplot=True)

for name, res in results.items():
    log_results(name, res)

2023-06-09 Bracket order placed: Entry 1866.34, SL 1881.41, TP 1843.74
2023-06-09 SELL EXECUTED, 1866.62
2023-06-13 BUY EXECUTED, 1902.58
2023-06-14 Bracket order placed: Entry 1883.03, SL 1900.14, TP 1857.36
2023-06-14 SELL EXECUTED, 1878.41
2023-06-16 BUY EXECUTED, 1900.14
2023-06-16 Bracket order placed: Entry 1872.16, SL 1893.16, TP 1840.67
2023-06-16 SELL EXECUTED, 1873.85
2023-06-23 BUY EXECUTED, 1834.00
2023-06-27 Bracket order placed: Entry 1841.75, SL 1825.45, TP 1866.20
2023-06-27 BUY EXECUTED, 1852.93
2023-06-29 SELL EXECUTED, 1882.29
2023-07-05 Bracket order placed: Entry 1877.96, SL 1892.97, TP 1855.44
2023-07-05 SELL EXECUTED, 1880.24
2023-07-06 BUY EXECUTED, 1830.76
2023-07-06 Bracket order placed: Entry 1867.13, SL 1886.06, TP 1838.72
2023-07-06 SELL EXECUTED, 1824.95
2023-07-06 BUY EXECUTED, 1830.37
2023-07-07 Bracket order placed: Entry 1862.51, SL 1839.99, TP 1896.27
2023-07-07 BUY EXECUTED, 1871.64
2023-07-11 SELL EXECUTED, 1909.07
2023-07-14 Bracket order placed: E